## 9- Temporal features feature-engineering
* Adding temporal features (based on __step__ column):
    * Lag features, e.g., steps since previous transaction, 
    * Rolling past N steps features, e.g., accumulated amount, etc.
* Assuming steps are time steps beginning from a certain point of time (hours or days).
* Will use iterative imputer for imputing missing temporal numerical feature values.

#### Stages

1. __Reading full data__.
2. __Checking if there are transactions within same time step__.
3. __Reading train/val/test datasets__.
4. __Adding temporal features__.
5. __Specifying column types__.
6. __Imputing missing values and One-hot-encoding categorical columns and standard scaling numerical columns__.
7. __Dropping irrelevant columns__.
8. __Saving data and preprocessor__.

#### General insights based on analysis:
* Time step values range from 0 to 180 -> assuming either hour or days
* There are overlapping transactions on a customer id level on the same time step:
    *  20,971 out of the total 594,643 records have transactions within the same time step

In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import StratifiedGroupKFold  # for splitting into train/val/test based on customer id while keeping fraud ratio the same for the three sets
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
# Enable experimental IterativeImputer (must be imported BEFORE the imputer)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
RANDOM_SEED = 420

In [3]:
PERCENTILES = [i * 0.025 for i in range(0, 41)]  # values between 0 and 1 in steps of 0.025

### 1- Reading full data

In [4]:
df = pd.read_csv("data/fraud_cleaned.csv")

In [5]:
df.shape

(594643, 10)

In [6]:
df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


### 2- Checking if there are transactions within same time step

In [7]:
df["step"].describe(percentiles=PERCENTILES)

count    594643.000000
mean         94.986827
std          51.053632
min           0.000000
0%            0.000000
2.5%          5.000000
5%           11.000000
7.5%         17.000000
10%          22.000000
12.5%        27.000000
15%          32.000000
17.5%        37.000000
20%          42.000000
22.5%        47.000000
25%          52.000000
27.5%        57.000000
30%          62.000000
32.5%        66.000000
35%          71.000000
37.5%        75.000000
40%          80.000000
42.5%        84.000000
45%          89.000000
47.5%        93.000000
50%          97.000000
52.5%       102.000000
55%         106.000000
57.5%       110.000000
60%         114.000000
62.5%       119.000000
65%         123.000000
67.5%       127.000000
70%         131.000000
72.5%       135.000000
75%         139.000000
77.5%       143.000000
80%         147.000000
82.5%       151.000000
85%         155.000000
87.5%       159.000000
90%         164.000000
92.5%       168.000000
95%         172.000000
97.5%      

In [8]:
customer_transactions_within_same_step_counts = df.groupby(['customer', 'step']).size().reset_index(name='txn_count').sort_values("txn_count", ascending=False)

In [9]:
customer_transactions_within_same_step_counts.head(20)

,customer,step,txn_count
571361,C997029022,163,5
546185,C910454738,153,5
86612,C1275518867,40,5
258520,C1849046345,20,5
513178,C806399525,71,5
258521,C1849046345,21,5
107873,C1350963410,47,5
511222,C800272493,39,5
513163,C806399525,50,5
354882,C245117318,170,5


In [10]:
customer_transactions_within_same_step_counts["txn_count"].describe(percentiles=PERCENTILES)

count    572133.000000
mean          1.039344
std           0.210097
min           1.000000
0%            1.000000
2.5%          1.000000
5%            1.000000
7.5%          1.000000
10%           1.000000
12.5%         1.000000
15%           1.000000
17.5%         1.000000
20%           1.000000
22.5%         1.000000
25%           1.000000
27.5%         1.000000
30%           1.000000
32.5%         1.000000
35%           1.000000
37.5%         1.000000
40%           1.000000
42.5%         1.000000
45%           1.000000
47.5%         1.000000
50%           1.000000
52.5%         1.000000
55%           1.000000
57.5%         1.000000
60%           1.000000
62.5%         1.000000
65%           1.000000
67.5%         1.000000
70%           1.000000
72.5%         1.000000
75%           1.000000
77.5%         1.000000
80%           1.000000
82.5%         1.000000
85%           1.000000
87.5%         1.000000
90%           1.000000
92.5%         1.000000
95%           1.000000
97.5%      

In [11]:
customer_transactions_within_same_step_counts.shape

(572133, 3)

In [12]:
customer_transactions_within_same_step_counts[customer_transactions_within_same_step_counts["txn_count"] > 1].shape

(20971, 3)

### 3- Reading train/val/test datasets

In [13]:
train_df = pd.read_csv("data/train_baseline_before_processing.csv")
val_df = pd.read_csv("data/validation_baseline_before_processing.csv")
test_df = pd.read_csv("data/test_baseline_before_processing.csv")

In [14]:
train_df.shape

(476608, 10)

In [15]:
train_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0
1,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0
2,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0
3,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0
4,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0


In [16]:
val_df.shape

(60291, 10)

In [17]:
val_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C39858251,5,F,28007,M348934600,28007,es_transportation,35.40,0
1,0,C1687101094,2,F,28007,M348934600,28007,es_transportation,19.31,0
2,0,C1622124632,2,M,28007,M348934600,28007,es_transportation,29.84,0
3,0,C187514477,3,M,28007,M348934600,28007,es_transportation,12.10,0
4,0,C1635613216,4,F,28007,M1053599405,28007,es_health,105.59,0


In [18]:
test_df.shape

(57744, 10)

In [19]:
test_df.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,C105845174,3,M,28007,M348934600,28007,es_transportation,32.40,0
1,0,C194016923,3,F,28007,M348934600,28007,es_transportation,30.19,0
2,0,C1207205377,4,M,28007,M1823072687,28007,es_transportation,17.54,0
3,0,C834963773,5,F,28007,M348934600,28007,es_transportation,40.69,0
4,0,C1425441042,2,M,28007,M1888755466,28007,es_otherservices,87.67,0


### 4- Adding temporal features

In [20]:
import numpy as np
import pandas as pd

def add_temporal_features(df, rolling_steps=(7, 30)):
    out = df.copy()
    out['_orig_order'] = np.arange(len(out))

    # Stable sort for time-aware ops
    out = out.sort_values(['customer', 'step'], kind='mergesort')

    # 1- Transaction-level cumulative (customer-only; strictly prior rows)
    # prior transaction count for this customer (0 for the first)
    out['cust_txn_count_prior_to_row'] = out.groupby('customer').cumcount()

    # prior cumulative sums
    out['cust_amount_sum_prior_to_row'] = (
        out.groupby('customer')['amount'].cumsum().shift(1).fillna(0.0)
    )

    # prior mean amount
    out['cust_amount_mean_prior_to_row'] = (
        out['cust_amount_sum_prior_to_row'] /
        out['cust_txn_count_prior_to_row'].replace(0, np.nan)
    ).fillna(0.0)

    # prior std amount via running sums of squares
    out['_amount_sq'] = out['amount'] ** 2
    S1_prior = out.groupby('customer')['amount'].cumsum().shift(1).fillna(0.0)
    S2_prior = out.groupby('customer')['_amount_sq'].cumsum().shift(1).fillna(0.0)
    n_prior = out.groupby('customer').cumcount().astype(float)
    var_prior = (S2_prior - (S1_prior ** 2) / n_prior.replace(0, np.nan)) / n_prior.replace(0, np.nan)
    out['cust_amount_std_prior_to_row'] = np.sqrt(var_prior.clip(lower=0)).fillna(0.0)
    out.drop(columns=['_amount_sq'], inplace=True)

    # days since previous transaction by this customer
    out['cust_days_since_previous_transaction'] = out['step'] - out.groupby('customer')['step'].shift(1)

    # convenience ratio vs customer’s prior mean
    eps = 1e-9
    out['amount_divided_by_cust_prior_mean'] = (
        out['amount'] / (out['cust_amount_mean_prior_to_row'] + eps)
    ).fillna(0.0)

    # 2- Per-(customer, day) table (unique index for rolling windows)
    daily = (
        out.groupby(['customer', 'step'], as_index=False)
           .agg(cust_day_total_transactions=('amount', 'size'),
                cust_day_total_amount=('amount', 'sum'))
           .sort_values(['customer', 'step'], kind='mergesort')
    )

    # gap to previous active day
    daily['cust_days_since_previous_active_day'] = (
        daily['step'] - daily.groupby('customer')['step'].shift(1)
    )

    # lifetime-to-date (prior days only)
    daily['cust_active_day_count_prior_to_day'] = daily.groupby('customer').cumcount()
    daily['cust_txn_count_prior_to_day'] = (
        daily.groupby('customer')['cust_day_total_transactions'].cumsum().shift(1).fillna(0).astype(int)
    )
    daily['cust_amount_sum_prior_to_day'] = (
        daily.groupby('customer')['cust_day_total_amount'].cumsum().shift(1).fillna(0.0)
    )
    daily['cust_prev_day_total_amount'] = (
        daily.groupby('customer')['cust_day_total_amount'].shift(1).fillna(0.0)
    )

    # 3- Rolling “last N days” on the daily table (exclude current day via shift(1))
    for win in rolling_steps:
        # counts over prior window of days
        daily[f'cust_total_transactions_last_{win}_days'] = (
            daily.groupby('customer')
                 .rolling(window=win, on='step')['cust_day_total_transactions']
                 .sum()
                 .reset_index(level=0, drop=True)
                 .shift(1)
                 .fillna(0)
                 .astype(int)
                 .to_numpy()
        )

        # amount sums over prior window of days
        daily[f'cust_total_amount_last_{win}_days'] = (
            daily.groupby('customer')
                 .rolling(window=win, on='step')['cust_day_total_amount']
                 .sum()
                 .reset_index(level=0, drop=True)
                 .shift(1)
                 .fillna(0.0)
                 .to_numpy()
        )

        # rolling mean / std of the customer's daily totals (prior-only baseline)
        daily[f'cust_day_amount_mean_last_{win}_days'] = (
            daily.groupby('customer')
                 .rolling(window=win, on='step')['cust_day_total_amount']
                 .mean()
                 .reset_index(level=0, drop=True)
                 .shift(1)
                 .to_numpy()
        )
        daily[f'cust_day_amount_std_last_{win}_days'] = (
            daily.groupby('customer')
                 .rolling(window=win, on='step')['cust_day_total_amount']
                 .std(ddof=0)
                 .reset_index(level=0, drop=True)
                 .shift(1)
                 .to_numpy()
        )

        # activity rhythm (mean day gap) in prior window
        daily[f'cust_mean_day_gap_last_{win}_days'] = (
            daily.groupby('customer')
                 .rolling(window=win, on='step')['cust_days_since_previous_active_day']
                 .mean()
                 .reset_index(level=0, drop=True)
                 .shift(1)
                 .to_numpy()
        )

    # 4- Merge daily features back to each transaction row
    keep_prefixes = (
        'cust_total_transactions_last_',
        'cust_total_amount_last_',
        'cust_day_amount_mean_last_',
        'cust_day_amount_std_last_',
        'cust_mean_day_gap_last_',
        'cust_active_day_count_prior_to_day',
        'cust_txn_count_prior_to_day',
        'cust_amount_sum_prior_to_day',
        'cust_prev_day_total_amount'
    )
    keep_cols = [
        'customer',
        'step',
        'cust_days_since_previous_active_day',
        'cust_day_total_transactions',
        'cust_day_total_amount'
    ] + [c for c in daily.columns if c.startswith(keep_prefixes)]
    
    out = out.merge(daily[keep_cols], on=['customer', 'step'], how='left')

    # 5- Per-transaction ratios vs recent customer baseline (prior-only)
    for win in rolling_steps:
        mu = out[f'cust_day_amount_mean_last_{win}_days']
        out[f'amount_divided_by_cust_day_mean_last_{win}_days'] = (
            out['amount'] / (mu.replace(0, np.nan) + eps)
        ).fillna(0.0)

    # Restore original order
    out = out.sort_values('_orig_order').drop(columns=['_orig_order'])
    return out

In [21]:
train = add_temporal_features(train_df, rolling_steps=(7, 30))
val = add_temporal_features(val_df, rolling_steps=(7, 30))
test = add_temporal_features(test_df, rolling_steps=(7, 30))

# Sanity checks: same rows, same order, only more columns
assert len(train) == len(train_df)
assert len(val)   == len(val_df)
assert len(test)  == len(test_df)

new_cols = sorted(set(train.columns) - set(train_df.columns))

In [22]:
new_cols

['amount_divided_by_cust_day_mean_last_30_days',
 'amount_divided_by_cust_day_mean_last_7_days',
 'amount_divided_by_cust_prior_mean',
 'cust_active_day_count_prior_to_day',
 'cust_amount_mean_prior_to_row',
 'cust_amount_std_prior_to_row',
 'cust_amount_sum_prior_to_day',
 'cust_amount_sum_prior_to_row',
 'cust_day_amount_mean_last_30_days',
 'cust_day_amount_mean_last_7_days',
 'cust_day_amount_std_last_30_days',
 'cust_day_amount_std_last_7_days',
 'cust_day_total_amount',
 'cust_day_total_transactions',
 'cust_days_since_previous_active_day',
 'cust_days_since_previous_transaction',
 'cust_mean_day_gap_last_30_days',
 'cust_mean_day_gap_last_7_days',
 'cust_prev_day_total_amount',
 'cust_total_amount_last_30_days',
 'cust_total_amount_last_7_days',
 'cust_total_transactions_last_30_days',
 'cust_total_transactions_last_7_days',
 'cust_txn_count_prior_to_day',
 'cust_txn_count_prior_to_row']

In [23]:
train.shape

(476608, 35)

In [24]:
train.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,cust_txn_count_prior_to_row,cust_amount_sum_prior_to_row,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_days_since_previous_transaction,amount_divided_by_cust_prior_mean,cust_days_since_previous_active_day,cust_day_total_transactions,cust_day_total_amount,cust_active_day_count_prior_to_day,cust_txn_count_prior_to_day,cust_amount_sum_prior_to_day,cust_prev_day_total_amount,cust_total_transactions_last_7_days,cust_total_amount_last_7_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_7_days,cust_mean_day_gap_last_7_days,cust_total_transactions_last_30_days,cust_total_amount_last_30_days,cust_day_amount_mean_last_30_days,cust_day_amount_std_last_30_days,cust_mean_day_gap_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_day_mean_last_30_days
24714,0,C1093826151,4,M,28007,M348934600,28007,es_transportation,4.55,0,0,5043.89,0.0,0.0,NaN,4.550000e+09,NaN,1,4.55,0,165,5043.89,0.0,7,284.35,40.621429,22.886801,1.0,30,960.51,32.017000,18.841246,1.133333,0.112010,0.142112
322129,0,C352968107,2,M,28007,M348934600,28007,es_transportation,39.68,0,0,5068.05,0.0,0.0,NaN,3.968000e+10,NaN,1,39.68,0,164,5068.05,0.0,7,157.80,22.542857,13.769846,1.0,30,743.26,24.775333,16.236783,1.066667,1.760203,1.601593
269033,0,C2054744914,4,F,28007,M1823072687,28007,es_transportation,26.89,0,0,5205.80,0.0,0.0,NaN,2.689000e+10,NaN,1,26.89,0,169,5205.80,0.0,7,101.74,14.534286,9.990580,1.0,32,1052.75,35.091667,33.142066,1.033333,1.850108,0.766279
194497,0,C1760612790,3,M,28007,M348934600,28007,es_transportation,17.25,0,0,4751.31,0.0,0.0,NaN,1.725000e+10,NaN,1,17.25,0,109,4751.31,0.0,7,180.79,25.827143,13.129432,1.0,31,1339.81,44.660333,63.223405,1.000000,0.667902,0.386249
415549,0,C757503768,5,M,28007,M348934600,28007,es_transportation,35.72,0,0,2880.04,0.0,0.0,NaN,3.572000e+10,NaN,1,35.72,0,89,2880.04,0.0,7,126.58,18.082857,8.229723,1.0,30,685.44,22.848000,15.186225,1.133333,1.975352,1.563375


In [25]:
val.shape

(60291, 35)

In [26]:
val.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,cust_txn_count_prior_to_row,cust_amount_sum_prior_to_row,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_days_since_previous_transaction,amount_divided_by_cust_prior_mean,cust_days_since_previous_active_day,cust_day_total_transactions,cust_day_total_amount,cust_active_day_count_prior_to_day,cust_txn_count_prior_to_day,cust_amount_sum_prior_to_day,cust_prev_day_total_amount,cust_total_transactions_last_7_days,cust_total_amount_last_7_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_7_days,cust_mean_day_gap_last_7_days,cust_total_transactions_last_30_days,cust_total_amount_last_30_days,cust_day_amount_mean_last_30_days,cust_day_amount_std_last_30_days,cust_mean_day_gap_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_day_mean_last_30_days
41768,0,C39858251,5,F,28007,M348934600,28007,es_transportation,35.40,0,0,5355.65,0.0,0.0,NaN,3.540000e+10,NaN,1,35.40,0,182,5355.65,0.0,9,562.88,80.411429,77.878849,1.000000,33,1257.56,41.918667,48.285875,1.033333,0.440236,0.844493
21862,0,C1687101094,2,F,28007,M348934600,28007,es_transportation,19.31,0,0,4866.41,0.0,0.0,NaN,1.931000e+10,NaN,1,19.31,0,168,4866.41,0.0,7,185.74,26.534286,15.046077,1.285714,30,750.09,25.003000,14.043403,1.133333,0.727738,0.772307
19618,0,C1622124632,2,M,28007,M348934600,28007,es_transportation,29.84,0,0,2859.05,0.0,0.0,NaN,2.984000e+10,NaN,1,29.84,0,64,2859.05,0.0,7,296.24,42.320000,21.109367,1.142857,32,1221.36,40.712000,37.659468,1.166667,0.705104,0.732953
28339,0,C187514477,3,M,28007,M348934600,28007,es_transportation,12.10,0,0,5477.04,0.0,0.0,NaN,1.210000e+10,NaN,1,12.10,0,180,5477.04,0.0,8,262.51,37.501429,28.525200,1.000000,35,1114.41,37.147000,33.190837,1.033333,0.322654,0.325733
20428,0,C1635613216,4,F,28007,M1053599405,28007,es_health,105.59,0,0,10347.57,0.0,0.0,NaN,1.055900e+11,NaN,1,105.59,0,177,10347.57,0.0,7,210.43,30.061429,19.897224,1.000000,34,1230.84,41.028000,41.831770,1.033333,3.512474,2.573608


In [27]:
test.shape

(57744, 35)

In [28]:
test.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud,cust_txn_count_prior_to_row,cust_amount_sum_prior_to_row,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_days_since_previous_transaction,amount_divided_by_cust_prior_mean,cust_days_since_previous_active_day,cust_day_total_transactions,cust_day_total_amount,cust_active_day_count_prior_to_day,cust_txn_count_prior_to_day,cust_amount_sum_prior_to_day,cust_prev_day_total_amount,cust_total_transactions_last_7_days,cust_total_amount_last_7_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_7_days,cust_mean_day_gap_last_7_days,cust_total_transactions_last_30_days,cust_total_amount_last_30_days,cust_day_amount_mean_last_30_days,cust_day_amount_std_last_30_days,cust_mean_day_gap_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_day_mean_last_30_days
2058,0,C105845174,3,M,28007,M348934600,28007,es_transportation,32.40,0,0,5165.16,0.0,0.0,NaN,3.240000e+10,NaN,1,32.40,0,173,5165.16,0.0,7,200.36,28.622857,8.662708,1.285714,31,832.51,27.750333,19.166959,1.066667,1.131962,1.167554
27995,0,C194016923,3,F,28007,M348934600,28007,es_transportation,30.19,0,0,6876.56,0.0,0.0,NaN,3.019000e+10,NaN,1,30.19,0,145,6876.56,0.0,10,334.47,47.781429,60.523828,2.142857,35,1611.74,53.724667,92.288274,1.300000,0.631835,0.561939
6416,0,C1207205377,4,M,28007,M1823072687,28007,es_transportation,17.54,0,0,6054.90,0.0,0.0,NaN,1.754000e+10,NaN,1,17.54,0,177,6054.90,0.0,8,567.65,81.092857,106.522144,1.142857,32,1446.07,48.202333,64.086922,1.033333,0.216295,0.363883
53393,0,C834963773,5,F,28007,M348934600,28007,es_transportation,40.69,0,0,5456.95,0.0,0.0,NaN,4.069000e+10,NaN,1,40.69,0,169,5456.95,0.0,7,254.41,36.344286,13.255256,1.000000,30,1077.31,35.910333,20.943916,1.066667,1.119571,1.133100
11822,0,C1425441042,2,M,28007,M1888755466,28007,es_otherservices,87.67,0,0,5122.39,0.0,0.0,NaN,8.767000e+10,NaN,1,87.67,0,167,5122.39,0.0,7,178.44,25.491429,17.350328,1.000000,30,708.80,23.626667,16.456457,1.166667,3.439195,3.710638


#### Checking number of nulls

In [29]:
train.isnull().sum()

step                                                 0
customer                                             0
age                                                  0
gender                                               0
zipcodeOri                                           0
merchant                                             0
zipMerchant                                          0
category                                             0
amount                                               0
fraud                                                0
cust_txn_count_prior_to_row                          0
cust_amount_sum_prior_to_row                         0
cust_amount_mean_prior_to_row                        0
cust_amount_std_prior_to_row                         0
cust_days_since_previous_transaction              3291
amount_divided_by_cust_prior_mean                    0
cust_days_since_previous_active_day               3332
cust_day_total_transactions                          0
cust_day_t

In [30]:
val.isnull().sum()

step                                                0
customer                                            0
age                                                 0
gender                                              0
zipcodeOri                                          0
merchant                                            0
zipMerchant                                         0
category                                            0
amount                                              0
fraud                                               0
cust_txn_count_prior_to_row                         0
cust_amount_sum_prior_to_row                        0
cust_amount_mean_prior_to_row                       0
cust_amount_std_prior_to_row                        0
cust_days_since_previous_transaction              413
amount_divided_by_cust_prior_mean                   0
cust_days_since_previous_active_day               416
cust_day_total_transactions                         0
cust_day_total_amount       

In [31]:
test.isnull().sum()

step                                                0
customer                                            0
age                                                 0
gender                                              0
zipcodeOri                                          0
merchant                                            0
zipMerchant                                         0
category                                            0
amount                                              0
fraud                                               0
cust_txn_count_prior_to_row                         0
cust_amount_sum_prior_to_row                        0
cust_amount_mean_prior_to_row                       0
cust_amount_std_prior_to_row                        0
cust_days_since_previous_transaction              408
amount_divided_by_cust_prior_mean                   0
cust_days_since_previous_active_day               412
cust_day_total_transactions                         0
cust_day_total_amount       

### Saving data before processing

In [32]:
train.to_csv("data/train_temporal_features_before_processing.csv", index=False)

In [33]:
val.to_csv("data/validation_temporal_features_before_processing.csv", index=False)

In [34]:
test.to_csv("data/test_temporal_features_before_processing.csv", index=False)

### 5- Specifying column types

In [35]:
target_column = "fraud"

In [36]:
categorical_columns_to_encode = [
    "age",
    "gender",
    "category"
]

In [37]:
numerical_columns = [
    "step",
    "amount",
    "amount_divided_by_cust_day_mean_last_30_days",
    "amount_divided_by_cust_day_mean_last_7_days",
    "amount_divided_by_cust_prior_mean",
    "cust_active_day_count_prior_to_day",
    "cust_amount_mean_prior_to_row",
    "cust_amount_std_prior_to_row",
    "cust_amount_sum_prior_to_day",
    "cust_amount_sum_prior_to_row",
    "cust_day_amount_mean_last_30_days",
    "cust_day_amount_mean_last_7_days",
    "cust_day_amount_std_last_30_days",
    "cust_day_amount_std_last_7_days",
    "cust_day_total_amount",
    "cust_day_total_transactions",
    "cust_days_since_previous_active_day",
    "cust_days_since_previous_transaction",
    "cust_mean_day_gap_last_30_days",
    "cust_mean_day_gap_last_7_days",
    "cust_prev_day_total_amount",
    "cust_total_amount_last_30_days",
    "cust_total_amount_last_7_days",
    "cust_total_transactions_last_30_days",
    "cust_total_transactions_last_7_days",
    "cust_txn_count_prior_to_day",
    "cust_txn_count_prior_to_row"
]

### 6- Imputing missing values and One-hot-encoding categorical columns and standard scaling numerical columns

__Notes__:
* I am prioritizing the ability to handle unknown values.
    * Setting __handle_unknown__ to "ignore" -> keep all columns as zeros when an unknown value is encountered
    * __drop__ must be set to None in this case which does not drop any value during one-hot-encoding.
    * There is concern for __multicollinearity issue__ for some models, but I am prioritizing handling unknown values in this case.
* Will use iterative imputer for imputing missing temporal numerical feature values.

In [38]:
numeric_pipe = Pipeline(steps=[
    # Iterative (MICE-like) imputation on numeric columns
    ("impute_iter",
     IterativeImputer(
         # estimator=None -> defaults to BayesianRidge (good baseline)
         max_iter=10, # number of imputation rounds
         initial_strategy="median",  # how the very first fill is done
         imputation_order="ascending",
         random_state=RANDOM_SEED
     )
    ),
    # Scale AFTER imputing
    ("scale", StandardScaler())
])

In [39]:
# In order to be able to pickle the processor object later as lambda functions cannot be picked
class ToString(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):   return X.astype("string")
    def set_output(self, *, transform=None): return self

In [40]:
categorical_pipe = Pipeline(steps=[
    # 1- Force uniform string dtype (prevents mixed-type categories)
    ("to_str", ToString()), # This cannot be picked -> FunctionTransformer(lambda X: X.astype("string")))
    # 2- One-hot-encoding (unknowns at transform time -> all zeros)
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop=None))
])

In [41]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numerical_columns),
        ("cat", categorical_pipe, categorical_columns_to_encode)
    ],
    verbose_feature_names_out=False,
    remainder="passthrough"
).set_output(transform="pandas")

In [42]:
%%time

# Fit on train set
preprocessor.fit(train)

CPU times: user 14min 22s, sys: 6.95 s, total: 14min 29s
Wall time: 1min 29s


,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,estimator,None
,missing_values,nan
,sample_posterior,False


In [43]:
%%time

train = preprocessor.transform(train)
val = preprocessor.transform(val)
test = preprocessor.transform(test)

CPU times: user 14.8 s, sys: 387 ms, total: 15.2 s
Wall time: 1.88 s


In [44]:
train.shape

(476608, 59)

In [45]:
train.head()

,step,amount,amount_divided_by_cust_day_mean_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_prior_mean,cust_active_day_count_prior_to_day,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_amount_sum_prior_to_day,cust_amount_sum_prior_to_row,cust_day_amount_mean_last_30_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_30_days,cust_day_amount_std_last_7_days,cust_day_total_amount,cust_day_total_transactions,cust_days_since_previous_active_day,cust_days_since_previous_transaction,cust_mean_day_gap_last_30_days,cust_mean_day_gap_last_7_days,cust_prev_day_total_amount,cust_total_amount_last_30_days,cust_total_amount_last_7_days,cust_total_transactions_last_30_days,cust_total_transactions_last_7_days,cust_txn_count_prior_to_day,cust_txn_count_prior_to_row,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_U,gender_E,gender_F,gender_M,gender_U,category_es_barsandrestaurants,category_es_contents,category_es_fashion,category_es_food,category_es_health,category_es_home,category_es_hotelservices,category_es_hyper,category_es_leisure,category_es_otherservices,category_es_sportsandtoys,category_es_tech,category_es_transportation,category_es_travel,category_es_wellnessandbeauty,customer,zipcodeOri,merchant,zipMerchant,fraud
24714,-1.858796,-0.303248,-0.336575,-0.389404,0.184301,-1.618242,-0.756986,-0.468526,0.785494,0.782420,0.076636,0.055546,-0.323900,-0.100796,-0.275646,-0.262811,0.184301,0.184301,0.217557,-0.164692,-0.320688,0.076004,0.053510,0.414661,0.025138,1.775647,-1.619035,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C1093826151,28007,M348934600,28007,0
322129,-1.858796,0.018468,0.379018,0.259294,1.780770,-1.618242,-0.756986,-0.468526,0.794361,0.791268,-0.127464,-0.253273,-0.347693,-0.194408,-0.042439,-0.262811,1.780769,1.780770,0.057332,-0.164692,-0.320688,-0.128219,-0.255846,0.414661,0.025138,1.754951,-1.619035,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C352968107,28007,M348934600,28007,0
269033,-1.858796,-0.098661,-0.030542,0.294679,1.199533,-1.618242,-0.756986,-0.468526,0.844915,0.841711,0.163293,-0.390076,-0.193254,-0.233214,-0.127344,-0.262811,1.199533,1.199533,-0.022781,-0.164692,-0.320688,0.162712,-0.392887,0.573166,0.025138,1.858431,-1.619035,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C2054744914,28007,M1823072687,28007,0
194497,-1.858796,-0.186943,-0.216873,-0.170615,0.761448,-1.618242,-0.756986,-0.468526,0.678118,0.675279,0.432978,-0.197171,0.081554,-0.200984,-0.191338,-0.262811,0.761447,0.761448,-0.102893,-0.164692,-0.320688,0.432559,-0.199646,0.493913,0.025138,0.616673,-1.619035,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C1760612790,28007,M348934600,28007,0
415549,-1.858796,-0.017797,0.360280,0.343973,1.600809,-1.618242,-0.756986,-0.468526,-0.008632,-0.009972,-0.181785,-0.329459,-0.357290,-0.251294,-0.068727,-0.262811,1.600809,1.600809,0.217557,-0.164692,-0.320688,-0.182572,-0.332164,0.414661,0.025138,0.202753,-1.619035,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C757503768,28007,M348934600,28007,0


In [46]:
val.shape

(60291, 59)

In [47]:
val.head()

,step,amount,amount_divided_by_cust_day_mean_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_prior_mean,cust_active_day_count_prior_to_day,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_amount_sum_prior_to_day,cust_amount_sum_prior_to_row,cust_day_amount_mean_last_30_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_30_days,cust_day_amount_std_last_7_days,cust_day_total_amount,cust_day_total_transactions,cust_days_since_previous_active_day,cust_days_since_previous_transaction,cust_mean_day_gap_last_30_days,cust_mean_day_gap_last_7_days,cust_prev_day_total_amount,cust_total_amount_last_30_days,cust_total_amount_last_7_days,cust_total_transactions_last_30_days,cust_total_transactions_last_7_days,cust_txn_count_prior_to_day,cust_txn_count_prior_to_row,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_U,gender_E,gender_F,gender_M,gender_U,category_es_barsandrestaurants,category_es_contents,category_es_fashion,category_es_food,category_es_health,category_es_home,category_es_hotelservices,category_es_hyper,category_es_leisure,category_es_otherservices,category_es_sportsandtoys,category_es_tech,category_es_transportation,category_es_travel,category_es_wellnessandbeauty,customer,zipcodeOri,merchant,zipMerchant,fraud
41768,-1.858796,-0.020728,0.007807,-0.260220,1.586267,-1.618242,-0.756986,-0.468526,0.899909,0.896585,0.355706,0.735242,-0.054908,0.463860,-0.070851,-0.262811,1.586266,1.586267,-0.022781,-0.164692,-0.320688,0.355241,0.734387,0.652418,1.254787,2.127478,-1.619035,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C39858251,28007,M348934600,28007,0
21862,-1.858796,-0.168078,-0.027586,-0.147065,0.855063,-1.618242,-0.756986,-0.468526,0.720360,0.717428,-0.121048,-0.185091,-0.367730,-0.181304,-0.177663,-0.262811,0.855063,0.855063,0.217557,0.101453,-0.320688,-0.121799,-0.187545,0.414661,0.025138,1.837735,-1.619035,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C1687101094,28007,M348934600,28007,0
19618,-1.858796,-0.071645,-0.046881,-0.155973,1.333595,-1.618242,-0.756986,-0.468526,-0.016335,-0.017659,0.321698,0.084561,-0.151986,-0.119046,-0.107761,-0.262811,1.333595,1.333595,0.297669,-0.031619,-0.320688,0.321212,0.082576,0.573166,0.025138,-0.314646,-1.619035,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C1622124632,28007,M348934600,28007,0
28339,-1.858796,-0.234106,-0.246544,-0.306498,0.527408,-1.618242,-0.756986,-0.468526,0.944459,0.941038,0.221221,0.002250,-0.192809,-0.042901,-0.225526,-0.262811,0.527407,0.527408,-0.022781,-0.164692,-0.320688,0.220675,0.000122,0.810923,0.639962,2.086087,-1.619035,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C187514477,28007,M348934600,28007,0
20428,-1.858796,0.622063,0.855604,0.948956,4.776022,-1.618242,-0.756986,-0.468526,2.731927,2.724605,0.330604,-0.124840,-0.113870,-0.131493,0.395099,-0.262811,4.776021,4.776022,-0.022781,-0.164692,-0.320688,0.330123,-0.127190,0.731670,0.025138,2.023999,-1.619035,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C1635613216,28007,M1053599405,28007,0


In [48]:
test.shape

(57744, 59)

In [49]:
test.head()

,step,amount,amount_divided_by_cust_day_mean_last_30_days,amount_divided_by_cust_day_mean_last_7_days,amount_divided_by_cust_prior_mean,cust_active_day_count_prior_to_day,cust_amount_mean_prior_to_row,cust_amount_std_prior_to_row,cust_amount_sum_prior_to_day,cust_amount_sum_prior_to_row,cust_day_amount_mean_last_30_days,cust_day_amount_mean_last_7_days,cust_day_amount_std_last_30_days,cust_day_amount_std_last_7_days,cust_day_total_amount,cust_day_total_transactions,cust_days_since_previous_active_day,cust_days_since_previous_transaction,cust_mean_day_gap_last_30_days,cust_mean_day_gap_last_7_days,cust_prev_day_total_amount,cust_total_amount_last_30_days,cust_total_amount_last_7_days,cust_total_transactions_last_30_days,cust_total_transactions_last_7_days,cust_txn_count_prior_to_day,cust_txn_count_prior_to_row,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_U,gender_E,gender_F,gender_M,gender_U,category_es_barsandrestaurants,category_es_contents,category_es_fashion,category_es_food,category_es_health,category_es_home,category_es_hotelservices,category_es_hyper,category_es_leisure,category_es_otherservices,category_es_sportsandtoys,category_es_tech,category_es_transportation,category_es_travel,category_es_wellnessandbeauty,customer,zipcodeOri,merchant,zipMerchant,fraud
2058,-1.858796,-0.048201,0.166206,0.012031,1.449933,-1.618242,-0.756986,-0.468526,0.830000,0.826829,-0.043617,-0.149414,-0.320924,-0.246848,-0.090766,-0.262811,1.449932,1.449933,0.057332,0.101453,-0.320688,-0.044321,-0.151806,0.493913,0.025138,1.941215,-1.619035,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C105845174,28007,M348934600,28007,0
27995,-1.858796,-0.068440,-0.130731,-0.184810,1.349501,-1.618242,-0.756986,-0.468526,1.458078,1.453536,0.688449,0.177854,0.347076,0.285659,-0.105437,-0.262811,1.349500,1.349501,0.618120,0.899887,-0.320688,0.688183,0.176031,0.810923,1.869611,1.361728,-1.619035,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C194016923,28007,M348934600,28007,0
6416,-1.858796,-0.184287,-0.227839,-0.348359,0.774627,-1.618242,-0.756986,-0.468526,1.156532,1.152648,0.532806,0.746882,0.089442,0.757968,-0.189413,-0.262811,0.774626,0.774627,-0.022781,-0.031619,-0.320688,0.532447,0.746048,0.573166,0.639962,2.023999,-1.619035,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C1207205377,28007,M1823072687,28007,0
53393,-1.858796,0.027718,0.149313,0.007153,1.826669,-1.618242,-0.756986,-0.468526,0.937086,0.933681,0.186366,-0.017516,-0.304691,-0.199692,-0.035734,-0.262811,1.826668,1.826669,0.057332,-0.164692,-0.320688,0.185800,-0.019679,0.414661,0.025138,1.858431,-1.619035,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,C834963773,28007,M348934600,28007,0
11822,-1.858796,0.457954,1.413097,0.920115,3.961655,-1.618242,-0.756986,-0.468526,0.814304,0.811167,-0.159839,-0.202905,-0.345686,-0.157644,0.276139,-0.262811,3.961654,3.961655,0.297669,-0.164692,-0.320688,-0.160613,-0.205391,0.414661,0.025138,1.817039,-1.619035,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,C1425441042,28007,M1888755466,28007,0


#### Checking number of nulls

In [50]:
train.isnull().sum()

step                                            0
amount                                          0
amount_divided_by_cust_day_mean_last_30_days    0
amount_divided_by_cust_day_mean_last_7_days     0
amount_divided_by_cust_prior_mean               0
cust_active_day_count_prior_to_day              0
cust_amount_mean_prior_to_row                   0
cust_amount_std_prior_to_row                    0
cust_amount_sum_prior_to_day                    0
cust_amount_sum_prior_to_row                    0
cust_day_amount_mean_last_30_days               0
cust_day_amount_mean_last_7_days                0
cust_day_amount_std_last_30_days                0
cust_day_amount_std_last_7_days                 0
cust_day_total_amount                           0
cust_day_total_transactions                     0
cust_days_since_previous_active_day             0
cust_days_since_previous_transaction            0
cust_mean_day_gap_last_30_days                  0
cust_mean_day_gap_last_7_days                   0


In [51]:
val.isnull().sum()

step                                            0
amount                                          0
amount_divided_by_cust_day_mean_last_30_days    0
amount_divided_by_cust_day_mean_last_7_days     0
amount_divided_by_cust_prior_mean               0
cust_active_day_count_prior_to_day              0
cust_amount_mean_prior_to_row                   0
cust_amount_std_prior_to_row                    0
cust_amount_sum_prior_to_day                    0
cust_amount_sum_prior_to_row                    0
cust_day_amount_mean_last_30_days               0
cust_day_amount_mean_last_7_days                0
cust_day_amount_std_last_30_days                0
cust_day_amount_std_last_7_days                 0
cust_day_total_amount                           0
cust_day_total_transactions                     0
cust_days_since_previous_active_day             0
cust_days_since_previous_transaction            0
cust_mean_day_gap_last_30_days                  0
cust_mean_day_gap_last_7_days                   0


In [52]:
test.isnull().sum()

step                                            0
amount                                          0
amount_divided_by_cust_day_mean_last_30_days    0
amount_divided_by_cust_day_mean_last_7_days     0
amount_divided_by_cust_prior_mean               0
cust_active_day_count_prior_to_day              0
cust_amount_mean_prior_to_row                   0
cust_amount_std_prior_to_row                    0
cust_amount_sum_prior_to_day                    0
cust_amount_sum_prior_to_row                    0
cust_day_amount_mean_last_30_days               0
cust_day_amount_mean_last_7_days                0
cust_day_amount_std_last_30_days                0
cust_day_amount_std_last_7_days                 0
cust_day_total_amount                           0
cust_day_total_transactions                     0
cust_days_since_previous_active_day             0
cust_days_since_previous_transaction            0
cust_mean_day_gap_last_30_days                  0
cust_mean_day_gap_last_7_days                   0


### 7- Dropping irrelevant columns

In [53]:
irrelevant_columns = [
    "customer",
    "zipcodeOri",
    "merchant",
    "zipMerchant"
]

In [54]:
train = train.drop(columns=irrelevant_columns)

In [55]:
train.shape

(476608, 55)

In [56]:
train.duplicated().sum()

np.int64(0)

In [57]:
val = val.drop(columns=irrelevant_columns)

In [58]:
val.shape

(60291, 55)

In [59]:
val.duplicated().sum()

np.int64(0)

In [60]:
test = test.drop(columns=irrelevant_columns)

In [61]:
test.shape

(57744, 55)

In [62]:
test.duplicated().sum()

np.int64(0)

### 8- Saving data and preprocessor

In [63]:
joblib.dump(preprocessor, "models/temporal_features_data_preprocessor.joblib")

['models/temporal_features_data_preprocessor.joblib']

In [64]:
train.to_csv("data/train_temporal_features.csv", index=False)

In [65]:
val.to_csv("data/validation_temporal_features.csv", index=False)

In [66]:
test.to_csv("data/test_temporal_features.csv", index=False)